In [15]:
import numpy as np
import stars
import matplotlib.pyplot as plt
import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
Component, Kinetics = stars.Fluid_definition_opt(2)
HR = 2.5 #[C/min or K/min]

"""
The parameters can be changed here, for example:
Component["WATER"].setCMM()
Kinetics["Cracking"].setEACT
"""
folder_path = './'
dat_file_name = 'test_try1'
newfile_path = folder_path + dat_file_name + '.dat'
fileID = open(newfile_path, 'a')
stars.print_IO_control(fileID, 2)
stars.print_grid(fileID, 2)
stars.print_fluid_opt(fileID, 2, Component, Kinetics)
stars.K_value_table(2, fileID)
stars.print_ref_cond(fileID, 2)
stars.print_rock_fluid(fileID, 2)
stars.print_initial_cond(fileID, 2)
stars.print_numerical(fileID, 2)
stars.print_recurrent(fileID, 2)
stars.print_heater(fileID, 2)
stars.print_heating_rate(fileID, HR, 2)
fileID.close()

In [ ]:
# Run CMG .dat file
exe_path = '"C:\\Program Files (x86)\\CMG\\STARS\\2017.10\\Win_x64\\EXE\\st201710.exe"'
cd_path = 'cd C:\\Users\\yunanli\\Desktop\\Tim_test '
stars.run_dat_file(exe_path, cd_path, dat_file_name)

# Read the results from CMG
(I separated these functions out of CMG_Python because we need to change SPEC-HIST values for different models)

In [ ]:
class RTO:
    def __init__(self, NAME, HR, TIME, TEMP, O2, X_O2, GRAD_X_O2):
        self.NAME = NAME
        self.HR = HR
        self.TIME = TIME
        self.TEMP = TEMP
        self.O2 = O2
        self.X_O2 = X_O2
        self.GRAD_X_O2 = GRAD_X_O2        
        
    def getNAME(self):
        return self.NAME
    def getHR(self):
        return self.HR
    def getTIME(self):
        return self.TIME
    def getTEMP(self):
        return self.TEMP
    def getO2(self):
        return self.O2
    def getX_O2(self):
        return self.X_O2
    def getGRAD_X_O2(self):
        return self.GRAD_X_O2    
    
    def setNAME(self, value):
        self.NAME = value
    def setHR(self, value):
        self.HR = value
    def setTIME(self, value):
        self.TIME = value
    def setTEMP(self, value):
        self.TEMP = value
    def setO2(self, value):
        self.O2 = value
    def setX_O2(self, value):
        self.X_O2 = value
    def setGRAD_X_O2(self, value):
        self.GRAD_X_O2 = value      
        
"""
Change the numSPHIST and self.SPEC_VALS according to dat files
"""
class STARScontainer():
    def __init__(self):
        self.numSPHIST = 22
        
    def parse_runfile(self, input_file_base, reaction_type = 'MURAT'):

        # Open STARS output files
        FID = [line.decode('utf-8', errors='ignore') for line in open(input_file_base + '.irf', 'rb')]
        FID_bin = open(input_file_base + '.mrf', 'rb')

        # Initialize variables/objects for parsing data
        i = 0
        self.UNIT = {} # Create units dictionary
        self.COMP = {} # Compositions dictionary
        self.TIME = {} # Time dictionary
        self.GRID = {} # Grid dictionary
        self.GRID['TEMP'] = []
        self.SP = {}
        self.SP['VAL'] = []
        self.SPHIST = {}

        self.TIME['VECT'] = []
        self.TIME['CHR'] = []
        self.TIME['CHR_UNIT'] = []

        self.REC = {}
        REC_list = ['WELL-REC', 'LAYER-REC', 'GROUP-REC', 'SECTOR-REC', 'RSTSPEC01-REC', 'RSTSPEC02-REC', 'RSTSPEC03-REC',
            'RSTSPEC04-REC', 'RSTSPEC05-REC', 'RSTSPEC06-REC', 'RSTSPEC07-REC', 'RSTSPEC08-REC', 'RSTSPEC09-REC',
            'RSTSPEC10-REC', 'RSTSPEC11-REC', 'RSTSPEC12-REC', 'RSTSPEC13-REC', 'RSTSPEC14-REC', 'RSTSPEC15-REC',
            'RSTSPEC16-REC', 'RSTSPEC17-REC', 'RSTSPEC18-REC', 'RSTSPEC19-REC', 'RSTSPEC20-REC', 'RSTSPEC21-REC',
            'RSTSPEC22-REC']

        skip_list = ['WELL-ARRAY', 'LAYER-ARRAY', 'GROUP-ARRAY']

        # Iterate over lines of file
        while i < len(FID):
            line_split = FID[i].split()

            # Parse case-by-case quantities
            if line_split[0] == 'INTERNAL-UNIT-TABLE':
                i+=1 
                self.UNIT['INT'] = [str(FID[i+j].split()[1]) for j in range(21)]
                self.UNIT['DESCP'] = [str(FID[i+j].split()[3]) for j in range(21)]
                i+=21 

            elif line_split[0] == 'OUTPUT-UNIT-TABLE':
                i+=1
                self.UNIT['OUT'] = [str(FID[i+j].split()[1]) for j in range(21)]
                i+=21

            elif line_split[0] == 'NCOMP':
                self.COMP['NUM'] = int(line_split[1])
                i+=1

            elif line_split[0] == 'COMPNAME':
                i+=1
                self.COMP['NAME'] = [str(FID[i+j].split()[1]).replace("'","") for j in range(self.COMP['NUM'])]
                i+=self.COMP['NUM']

            elif line_split[0] == 'COMP-PHASE-TEMPLATE':
                i+=1
                self.COMP['TEMPL'] = [str(FID[i+j].split()[2]) for j in range(self.COMP['NUM'])]
                i+=self.COMP['NUM']

            elif line_split[0] == 'TIME' and 'SPEC-HISTORY' in [FID[i-1].split()[0], FID[i-3].split()[0]]:
                self.TIME['VECT'].append(line_split[1:])
                i+=1

            elif line_split[0] == 'TIMCHR':
                self.TIME['CHR'].append(line_split[2])
                self.TIME['CHR_UNIT'].append(line_split[3].replace("'",""))
                i+=1


            # Parse grid values from binary file
            elif line_split[0] == 'GRID':
                item_num = int(FID[i].split()[2])
                for j in range(item_num):
                    num_bytes = np.fromfile(FID_bin, np.int64, count=1).byteswap()
                    _ = np.fromfile(FID_bin, np.int8, count=np.asscalar(num_bytes)).byteswap() 
                _, i = self.parse_nobin(FID, i)

            elif line_split[0] == 'GRID-VALUE':
                props_list, i = self.parse_nobin(FID, i)
                for prop in props_list[3:]:
                    num_bytes = np.fromfile(FID_bin, np.int64, count=1).byteswap()
                    if prop == 'TEMP':
                        grid_temp = np.fromfile(FID_bin, np.float64, count=int(num_bytes/8)).byteswap()
                        self.GRID['TEMP'].append(grid_temp)
                    else:
                        _ = np.fromfile(FID_bin, np.int8, count=np.asscalar(num_bytes)).byteswap()


            # Parse species names and values from binary file
            elif line_split[0] == 'SPHIST-NAMES':
                self.SPHIST['NUM'] = []
                self.SPHIST['NAME'] = []
                i+=1
                for j in range(self.numSPHIST):
                    line_split = FID[i+j].split()
                    self.SPHIST['NUM'].append(line_split[0])
                    self.SPHIST['NAME'].append(' '.join([line.replace("'","") for line in line_split[3:]]))
                i+=self.numSPHIST

            elif line_split[0] == 'SPEC-HISTORY':
                props_list, i = self.parse_nobin(FID, i)
                for prop in props_list[3:]:
                    num_bytes = np.fromfile(FID_bin, np.int64, count=1).byteswap()
                    # print(prop)
                    # print(np.asscalar(num_bytes))
                    if prop == 'SPVALS':
                        spvals_temp = np.fromfile(FID_bin, np.float64, count=self.numSPHIST).byteswap()
                        self.SP['VAL'].append(spvals_temp)
                    else:
                        _ = np.fromfile(FID_bin, np.byte, count=np.asscalar(num_bytes)).byteswap()


            # Variables stored in binary but skipped in parsing
            elif line_split[0] in skip_list:
                item_num = int(line_split[2])
                for j in range(item_num):
                    num_bytes = np.fromfile(FID_bin, np.int64, count=1).byteswap()
                    _ = np.fromfile(FID_bin, np.byte, count=np.asscalar(num_bytes)).byteswap() 
                i+=1


            # Other variables parsed from text file
            elif line_split[0] in REC_list:
                list_temp = FID[i].split()
                while list_temp[-1] != '/':
                    i+=1
                    list_temp += FID[i].split()
                self.REC[line_split[0]] = list_temp[1:-1]
                i+=1

            else:
                i+=1

        FID_bin.close()

        # OUTPUT VARIABLES MURAT
        self.t = (np.asarray(self.TIME['VECT'])[:,1]).astype(np.float)
        TEMP_VALS = np.asarray(self.GRID['TEMP'])
        self.lin_HR = np.asarray(TEMP_VALS)[:,8]
        self.SPEC_VALS = np.asarray(self.SP['VAL'])
        

        if reaction_type == 'MURAT':
            #self.N2_PROD   = self.SPEC_VALS[:,8]
            self.O2_PROD   = self.SPEC_VALS[:,8] 
            self.TEMP_avg   = self.SPEC_VALS[:,21]
            #self.TEMP_113 = self.SPEC_VALS[:,17]
            #self.H2O_PROD  = self.SPEC_VALS[:,10]
            #self.CO_PROD   = self.SPEC_VALS[:,11]
            #self.CO2_PROD  = self.SPEC_VALS[:,12]
        elif reaction_type == 'BO':
            # OUTPUT VARIABLES BO
            self.N2_PROD = self.SPEC_VALS[:,6]
            self.O2_PROD = self.SPEC_VALS[:,7] 
            self.H2O_PROD = self.SPEC_VALS[:,8]
            self.CO_PROD = self.SPEC_VALS[:,9]
            self.CO2_PROD = self.SPEC_VALS[:,10]
            
    @staticmethod
    def parse_nobin(file_in, i):
        '''
        file_in - file to be parsed
        i - current line number
        '''
        
        list_temp = file_in[i].split()
        while list_temp[-1] != '/':
            i+=1
            list_temp += file_in[i].split()
        i+=1

        return list_temp[1:-1], i

In [ ]:
RTO_result = dict()
container = STARScontainer()
container.parse_runfile(folder_path + dat_file_name)
TIME = 24*60*container.t #unit is day => min
TEMP = container.TEMP_avg - 273.15 # unit is [K] => [C]
MFO2 = container.O2_PROD
RTO_result[dat_file_name].setTIME(TIME)
RTO_result[dat_file_name].setTEMP(TEMP)
RTO_result[dat_file_name].setO2(MFO2)
O2_react = max(MFO2)-np.asarray(MFO2)
O2_interval = np.multiply(O2_react, np.asarray(TIME))
O2_integ = np.zeros(len(O2_interval))
dO2_conversion_dt = np.zeros(len(O2_interval))
for i in range(len(O2_interval)):
    O2_integ[i] = sum(O2_interval[0:(i+1)]) 
O2_conversion = O2_integ/O2_integ[-1]
dO2_conversion_dt[1:] = np.diff(O2_conversion)/np.diff(TIME)
RTO_result[dat_file_name].setX_O2(O2_conversion.tolist())
RTO_result[dat_file_name].setGRAD_X_O2(dO2_conversion_dt.tolist())

In [ ]:
plt.plot(RTO_result[dat_file_name].getTIME(), RTO_result[dat_file_name].getTEMP())
plt.xlabel('Time, min')
plt.ylabel('Temperature, C')
plt.title('Temp VS Time')
plt.grid()
plt.show()
    
plt.plot(RTO_result[dat_file_name].getTEMP(), RTO_result[dat_file_name].getMFO2())
plt.xlabel('Temperature, C')
plt.ylabel('O2 Concentration')
plt.title('Concentration vs temp')
#plt.legend(loc = 'best')
plt.grid()
plt.show()